In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import json
import re
import string

from nltk.corpus import stopwords

In [2]:
def to_dict(string):
    if string != "[]":
        string = json.loads(string.replace("'", "\""))
        return ",".join([s["screen_name"] for s in string])
    return ""

def to_list(list_):
    if list_ != "[]":
        list_ = list_[1:-1]
        list_ = list_.split(",")
        return ",".join([s.strip().strip("'") for s in list_])
    return ""

In [3]:
def to_dict(string):
    if string != "[]":
        string = json.loads(string.replace("'", "\""))
        return ",".join([s["screen_name"] for s in string])
    return ""

def to_list(list_):
    if list_ != "[]":
        list_ = list_[1:-1]
        list_ = list_.split(",")
        return ",".join([s.strip().strip("'") for s in list_])
    return ""

def normalize(s):
    replacements = (("á", "a"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"))
    for a, b in replacements:
        s = s.lower()
        s = s.replace(a, b)
    return s

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r"", text)

def cleanTxt(text):
    text = re.sub(r"@[a-zA-Z0-9]+", "", text) #Removes @mentions
    text = re.sub(r"#", "", text) #Removing the "#" symbol
    text = re.sub(r"RT[\s]+", "", text) #Removing RT
    text = re.sub(r"https?:\/\/\S+", "", text) #Remove the hyperlink
    return text

def replace_punct(s):
    for i in string.punctuation:
        if i in s:
            s = s.replace(i, "").strip()
    return s

def replace_num(s):
    for i in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
        s = s.replace(i, "")
    return s

def preprocessor(text):
    text = re.sub(r"[\W]+", "", text.lower()) 
    return text

def tokenizador(text):
    important_words = []
    for word in text.split(" "):
        if word not in stopwords.words("spanish"):
            if word != "":
                important_words.append(word)
    return " ".join(important_words).strip()

In [4]:
df = pd.read_csv("C:/Users/Daniel/Desktop/proyecto/notebooks/tweets_periodicos.csv")
df.drop(df.columns[0], axis = 1, inplace = True)

In [5]:
columns_to_drop = ["conversation_id", "cashtags", "timezone", "user_id", "name", "thumbnail", "created_at", "id", "link"]

df.drop(columns_to_drop, axis = 1, inplace = True)

df = df[df.language == "es"]

df.drop("language", axis = 1, inplace = True)

df = df.reset_index().drop("index", axis = 1)

In [6]:
df.photos = df.photos.apply(lambda x : 1 if x != "[]" else 0)
df.retweet = df.retweet.apply(lambda x : 1 if x == "True" else 0)
df.urls = df.urls.apply(lambda x : 1 if x != "[]" else 0)
df.date = df.date.apply(lambda x : datetime.strptime(x, "%Y-%m-%d"))
df.time = df.time.apply(lambda x : datetime.strptime(x, "%H:%M:%S"))

In [7]:
for col in df.columns:
    if df[col].dtype == "int64":
        df[col] = df[col].astype("int32")
        
reply_to_rows = []
for num, row in enumerate(df.reply_to):
    try:
        to_dict(row)
    except:
        reply_to_rows.append(num)
        
df.drop(reply_to_rows, inplace = True)

df.reply_to = df.reply_to.apply(to_dict)

df = df.reset_index().drop("index", axis = 1)

In [8]:
mention_rows = []
for num, row in enumerate(df.mentions):
    try:
        to_dict(row)
    except:
        mention_rows.append(num)
        
df.drop(mention_rows, inplace = True)

df.mentions = df.mentions.apply(to_dict)

df = df.reset_index().drop("index", axis = 1)

In [9]:
hashtags_rows = []
for num, row in enumerate(df.hashtags):
    try:
        to_list(row)
    except:
        hashtags_rows.append(num)
        
df.drop(hashtags_rows, inplace = True)

df.hashtags = df.hashtags.apply(to_list)

df = df.reset_index().drop("index", axis = 1)

In [10]:
df.tweet = df.tweet.apply(normalize)
df.tweet = df.tweet.apply(deEmojify)
df.tweet = df.tweet.apply(cleanTxt)
df.tweet = df.tweet.apply(replace_punct)
df.tweet = df.tweet.apply(replace_num)

df.tweet = df.tweet.apply(tokenizador)

In [11]:
df.head()

,date,time,username,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,retweet,video,reply_to
0,2020-04-30,1900-01-01 23:59:01,elmundoes,gobierno regula navegacion recreativa puede ha...,,1,0,13,32,61,,0,0,
1,2020-04-30,1900-01-01 23:08:53,elmundoes,afrontar desconfinamiento segun psicologo javi...,,0,0,4,8,22,,0,1,
2,2020-04-30,1900-01-01 23:01:01,elmundoes,portada mundo viernes mayo disponible orbyt ca...,,0,1,11,68,74,,0,1,
3,2020-04-30,1900-01-01 22:42:01,elmundoes,cinco problemas piel provoca coronavirus,,1,0,5,65,47,,0,0,
4,2020-04-30,1900-01-01 22:41:20,elmundoes,madrid subvencionara autonomos pymes agilizara...,,1,0,13,98,328,,0,0,
